In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [18]:
data = pd.read_csv('tipcars.csv', delimiter=';')

In [19]:
data.isnull().sum()

id              0
znacka          0
model          46
vyrobeno      119
stav         3095
tachometr     696
objem         527
vykon          39
palivo          0
karoserie     136
barva         191
metaliza      191
typ             0
cena            0
dtype: int64

In [20]:
data = data.dropna(subset=['model']) # Odstranění řádků, kde je prázdný atribut 'model'

In [21]:
# Převod eurů na koruny
def eur_czk(cena):
    if 'EUR' in cena:
        cena = cena.replace('EUR', '').replace(' ', '').strip()
        return float(cena) * 25
    else:
        return float(cena.replace(' ', ''))

data['cena'] = data['cena'].apply(eur_czk) 

In [22]:
print(data.dtypes) # Oveření datových typu 

id             int64
znacka        object
model         object
vyrobeno     float64
stav          object
tachometr    float64
objem        float64
vykon        float64
palivo        object
karoserie     object
barva         object
metaliza      object
typ           object
cena         float64
dtype: object


In [23]:
# Funkce pro vypočet průměrného roku jednotlivých stavů
def compute_mean_years(data, column, condition_column):
    unique_conditions = data[condition_column].unique()
    mean_years = {}
    
    for condition in unique_conditions:
        condition_data = data[(data[condition_column] == condition) & (~data[column].isnull())]
        mean_value = condition_data[column].mean()
        
        if not np.isnan(mean_value):
            mean_years[condition] = round(mean_value)
        else:
            mean_years[condition] = None
        
    return mean_years

mean_years_by_condition = compute_mean_years(data, 'vyrobeno', 'stav')

In [24]:
# Funkce pro vyplnění hodnot u atributu vyrobeno podle stavu
def fillna_based_on_condition(data, column, condition_column, mean_values):
    missing_rows = data[column].isnull()
    
    for index, row in data[missing_rows].iterrows():
        condition = row[condition_column]
        data.at[index, column] = mean_values[condition]

fillna_based_on_condition(data, 'vyrobeno', 'stav', mean_years_by_condition)

In [25]:
# Funkce pro vypočet nejbližšího stavu podle roku
def find_closest_category(year, mean_years):
    closest_condition = None
    closest_difference = float('inf')
    
    for condition, mean_year in mean_years.items():
        if mean_year is not None:
            difference = abs(year - mean_year)
            if difference < closest_difference:
                closest_difference = difference
                closest_condition = condition
                
    return closest_condition

In [26]:
# Vyplnění hodnot u atributu stav
def fillna_state_based_on_year(data, column, condition_column, mean_values):
    missing_rows = data[column].isnull()
    
    for index, row in data[missing_rows].iterrows():
        year = row[condition_column]
        closest_condition = find_closest_category(year, mean_values)
        data.at[index, column] = closest_condition
        
fillna_state_based_on_year(data, 'stav', 'vyrobeno', mean_years_by_condition)

In [27]:
# Pokud auto má stav nové vozidlo bude typ převeden na nové
def update_type(row):
    if row['stav'] == 'nové vozidlo':
        row['typ'] = 'nové'
    return row

data = data.apply(update_type, axis=1)

In [28]:
data.loc[(data['tachometr'].isnull()) & (data['typ'] == 'nové'), 'tachometr'] = 0 # Pokud auto v typu nové tachometr bude 0

In [29]:
# Funkce pro vypočet objemu a výkonu podle značky, modelu a roku vyroby auta
def compute_mean_values(data, column, groupby_columns):
    grouped_data = data.groupby(groupby_columns).agg({column: 'mean'}).reset_index()

    mean_values = {}
    for index, row in grouped_data.iterrows():
        brand_model_year_tuple = tuple(row[groupby_columns])
        if not np.isnan(row[column]):
            mean_values[brand_model_year_tuple] = round(row[column])
        else:
            mean_values[brand_model_year_tuple] = np.nan

    return mean_values

mean_vykon = compute_mean_values(data, 'vykon', ['znacka', 'model', 'vyrobeno'])
mean_objem = compute_mean_values(data, 'objem', ['znacka', 'model', 'vyrobeno'])

In [30]:
# Funkce pro vyplnění chybejicích hodnot
def fillna_based_on_brand_model_and_year(data, column, groupby_columns, mean_values):
    missing_rows = data[column].isnull()

    for index, row in data[missing_rows].iterrows():
        if row['palivo'] != 'elektro':
            brand_model_year_tuple = tuple(row[groupby_columns])

            if brand_model_year_tuple in mean_values:
                data.at[index, column] = mean_values[brand_model_year_tuple]

# Vyplnění chybejicích hodnot u atributů vykon a objem
fillna_based_on_brand_model_and_year(data, 'vykon', ['znacka', 'model', 'vyrobeno'], mean_vykon) #
fillna_based_on_brand_model_and_year(data, 'objem', ['znacka', 'model', 'vyrobeno'], mean_objem) #

In [31]:
# Funkce pro vypočet mody pro atribut karoserie
def compute_mode_values(data, column, groupby_columns):
    grouped_data = data.groupby(groupby_columns)[column].agg(pd.Series.mode).reset_index()
    mode_values = {}

    for _, row in grouped_data.iterrows():
        key = tuple(row[groupby_columns].values)
        mode = row[column]
        if isinstance(mode, pd.Series) or isinstance(mode, np.ndarray):
            mode = mode[0] if len(mode) > 0 else None
        mode_values[key] = mode

    return mode_values

mode_karoserie = compute_mode_values(data, 'karoserie', ['znacka', 'model']) #

In [32]:
# Funkce pro vyplnění chybejicích hodnot u atrubutu karoserie
def fillna_using_modes(row, column, mode_values, groupby_columns):
    if pd.isna(row[column]):
        key = tuple(row[groupby_columns].values)
        if key in mode_values:
            return mode_values[key]
    return row[column]

data['karoserie'] = data.apply(lambda row: fillna_using_modes(row, 'karoserie', mode_karoserie, ['znacka', 'model']), axis=1)

In [33]:
# Vylpnění hodnot u atributu barva podle distribuci v datasetu
color_distribution = data['barva'].value_counts(normalize=True)
colors = color_distribution.index
color_probabilities = color_distribution.values

data.loc[data['barva'].isnull(), 'barva'] = np.random.choice(colors, size=data['barva'].isnull().sum(),p=color_probabilities)

In [34]:
# Vylpnění hodnot u atributu metaliza podle distribuci v datasetu
metaliza_distribution = data['metaliza'].value_counts(normalize=True)
metaliza_values = metaliza_distribution.index
metaliza_probabilities = metaliza_distribution.values

data.loc[data['metaliza'].isnull(), 'metaliza'] = np.random.choice(metaliza_values, size=data['metaliza'].isnull().sum(), p=metaliza_probabilities)
#

In [35]:
# Odstranění zaznamů, které stalé mají chybejicí hodnoty
data = data.dropna(subset=['tachometr'])
data = data.dropna(subset=['vykon'])
data = data[~((data['palivo'] != 'elektro') & (data['objem'].isnull()))]

In [36]:
# Nastavení horní a dolní hranicí pro odstranění anomalií
upper_bound_vykon = 610
data = data[(data['vykon'] <= upper_bound_vykon)]

lower_bound_objem = 400
upper_bound_objem = 10000
data = data[((data['objem'] >= lower_bound_objem) & (data['objem'] < upper_bound_objem)) | (pd.isna(data['objem']))]

data = data[data['tachometr'] != 1752128]

upper_bound_cena = 20000000
data = data[(data['cena'] < upper_bound_cena)]

In [174]:
data.to_csv('tipcars_final.csv')

In [37]:
len(data)

17932